In [1]:
# Loading in all packages from the jupyter environment
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import seaborn as sns
import subprocess as sp
import itertools
import ipywidgets

In [2]:
Targetpaths_file="/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/lako2021/ANANSE/target_files.csv"
output_dir="/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/lako2021/ANANSE/"
Path(f"{output_dir}/targetscsv").mkdir(parents=True, exist_ok=True)
Path(f"{output_dir}/targetscsv_abs").mkdir(parents=True, exist_ok=True)
Path(f"{output_dir}/targetscsv_top").mkdir(parents=True, exist_ok=True)
#Targetpaths_file.files.loc["genome_path_size", :]

In [3]:
#Lets loop over all the comparisons Ananse needs to make:
sample_data = pd.read_table(Targetpaths_file, 
                            sep = ',', comment = '#')
#sample_data = sample_data.iloc[[0,1],:]

#sample_data=sample_data[3:10] use this for smaller populations comparison

sample_data

,cell_id,diffnetwork
0,MEC,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data...
1,CSSCs,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data...
2,LPCs,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data...
3,LNPCs,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data...
4,CB,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data...
5,CSB,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data...
6,CjS,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data...
7,StC,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data...


In [18]:
# List of target genes
topgenes = ['EHF','ETV2','FOSL2','GRHL1','ELF3','ZBTB33','ELK4','PAX6','USF1','DPB','KLF5','GRHL2']
edf = None


for index, cell_type in sample_data.iterrows():
    cell_id = sample_data.iloc[index,0]
    print(cell_id)
    target_file = sample_data.iloc[index,1]
    
    if not os.path.exists(f"{output_dir}/targetscsv_top/{cell_id}top.tsv"): 
        target_data = pd.read_table(target_file, sep = '\t', comment = '#', 
                                header=None, names = ['TF','target','weight','source_weight',
                                                      'target_weight','tf_expr_source',
                                                      'tf_expr_target','tg_expr_source','tg_expr_target','source_wb','target_wb','source_Tf_act','target_Tf_act'])
        target_data = target_data[['TF','target','weight','source_wb','target_wb']]
        # remove the first row where the header is double located
        target_data = target_data.iloc[1: , :]
        target_data["source_wb"] = target_data["source_wb"].astype(float)
        target_data["target_wb"] = target_data["target_wb"].astype(float)
        target_data['diff_wb'] = target_data['target_wb']- target_data["source_wb"]
    
        target_data = target_data[['TF','target','weight','diff_wb']]
        print("Generating the diff_wb tsv file")
        target_data.to_csv(
        f"{output_dir}/targetscsv/{cell_id}weightdiffwb.tsv",
        sep="\t",
        header=True,
        index=True)
    
        # Getting an absolute value according to weight times diff_wb
        target_data["diff_wb"] = target_data["diff_wb"].astype(float)
        target_data["weight"] = target_data["weight"].astype(float)
        target_data['abs_wb'] = target_data['weight']* target_data["diff_wb"]
        target_data = target_data[['TF','target','abs_wb']]
        print("Generating the abs_wb tsv file")
        target_data.to_csv(
        f"{output_dir}/targetscsv_abs/{cell_id}weightabswb.tsv",
        sep="\t",
        header=True,
        index=True)
    
        # Selecting the top target genes based on a list of TFs of interest
        target_data = target_data.set_index('TF')
        print("Selecting top targets from topgenes")
        target_top = target_data.loc[target_data.index.intersection(topgenes),:]
        print("Ordering genes according to score")
        target_top = target_top.sort_values(by=['abs_wb'], ascending=False)
        a = target_top['target'].unique()
        print(a)
        a = pd.DataFrame(a[1:100])
        print("Generating tsv files of top 100 target genes")
        a.to_csv(
        f"{output_dir}/targetscsv_top/{cell_id}top.tsv",
        sep="\t",
        header=False,
        index=False)

MEC
CSSCs
LPCs
LNPCs
CB
CSB
Generating the diff_wb tsv file
Generating the abs_wb tsv file
Selecting top targets from topgenes
Ordering genes according to score
['ARHGAP40' 'SNORC' 'CEMIP2' ... 'OSR1' 'STOX2' 'TPBG']
Generating tsv files of top 100 target genes
CjS
StC


In [ ]:
# Concatenating all cell files to one final file
import pandas as pd
import glob

path = f"{output_dir}/targetscsv_top/"
all_files = []

for index, cell_type in sample_data.iterrows():
    cell_id = sample_data.iloc[index,0]
    print(cell_id)
    all_files = glob.glob(path + f"*top.tsv")

print("Joining the individual top 100 target gene list to one dataframe")

if not os.path.exists(f"{output_dir}/targetscsv_top/joinedtargettable.tsv"):

    # Joining all files to one dataframe
    dffull = pd.concat((pd.read_table(f) for f in all_files),axis=1)

    # Merging the loc columns to one
    dffull = dffull.groupby(level=0, axis=1).first()

    # Setting loc as the index
    dffull.index = dffull['loc']
    del dffull['loc']

    dffull.to_csv(
                f"{output_dir}/targetscsv_top/joinedtargettable.tsv",
                sep="\t",
                header=True,
                index=True,
            )

    dffull